In [3]:
import monkdata as m
import numpy as np
import dtree as dt
import pandas as pd
import drawtree_qt5 as drawt

## Assignment 1

In [4]:
datasets = [m.monk1, m.monk2, m.monk3]
ent = np.zeros(len(datasets))
for i, v in enumerate(datasets):
    ent[i] = dt.entropy(v)
print(ent)

[1.         0.95711743 0.99980613]


## Assignment 2

In [32]:
np.random.seed(123)
uset = np.random.binomial(1, .5, 1000)
p_u = sum(uset)/len(uset)
print(p_u)
ent_uset = -p_u*np.log2(p_u) - (1-p_u)*np.log2(1-p_u)
print(ent_uset)

0.492
0.9998153271549207


In [33]:
eset = np.random.binomial(1, .1, 1000)
p_e = sum(eset)/len(eset)
print(p_e)
ent_eset = -p_e*np.log2(p_e) - (1-p_e)*np.log2(1-p_e)
print(ent_eset)

0.106
0.48773158354050183


## Assignment 3

In [5]:
len_data = len(datasets)
len_att = len(m.attributes)
info = np.zeros((len_data, len_att))
for i, v in enumerate(datasets):
    for j in range(len(m.attributes)):
        info[i,j] = dt.averageGain(v, m.attributes[j])
print(info)

[[0.07527256 0.00583843 0.00470757 0.0263117  0.28703075 0.00075786]
 [0.00375618 0.0024585  0.00105615 0.01566425 0.01727718 0.00624762]
 [0.00712087 0.29373617 0.00083111 0.00289182 0.25591172 0.00707703]]


In [65]:
roots = np.argmax(info, axis = 1)
print(roots)

[4 4 1]


## Assignment 4

In [66]:
subent = np.zeros((len_data, 4))
for i, x in enumerate(datasets):
    att = m.attributes[roots[i]]
    for j, v in enumerate(att.values):
        subent[i,j] = dt.entropy(dt.select(x, att, v))
print(subent)

[[0.         0.93831535 0.94807824 0.90817835]
 [0.91034806 1.         0.96333555 0.877962  ]
 [0.91829583 0.8296071  0.37764632 0.        ]]


## Assignment 5

In [67]:
nodes_1 = np.empty((len_data, 4))
for i, x in enumerate(datasets):
    att = m.attributes[roots[i]]
    for j, v in enumerate(att.values):
        subsets = dt.select(x, att, v)
        info = np.zeros(len_att)
        for k in range(len_att):
            info[k] = dt.averageGain(subsets, m.attributes[k])
        nodes_1[i,j] = np.argmax(info)
print(nodes_1)

# mostCommon to find the outcome (+ or -) of each leaf

[[0. 3. 5. 0.]
 [2. 2. 2. 1.]
 [4. 4. 3. 0.]]


In [ ]:
#drawt.drawTree(dt.buildTree(m.monk1, m.attributes, 2))
#drawt.drawTree(dt.buildTree(m.monk2, m.attributes, 2))
drawt.drawTree(dt.buildTree(m.monk3, m.attributes, 2))

In [8]:
testsets = [m.monk1test, m.monk2test, m.monk3test]
err = np.zeros((len_data, 2))
for i, x in enumerate(datasets):
    t = dt.buildTree(x, m.attributes)
    err[i, 0] = 1 - dt.check(t, x)
    err[i, 1] = 1 - dt.check(t, testsets[i])
print(err)

[[0.         0.1712963 ]
 [0.         0.30787037]
 [0.         0.05555556]]
